In [4]:
%%capture
#!pip install  "xarray[complete]" dask-expr dask
!pip install --upgrade parallel-pandas

#!pip install --upgrade git+https://github.com/pangeo-data/xESMF.git

In [1]:
# Importing libraries
import xarray as xr
import os
import pandas as pd
import numpy as np
import pandas as pd
from dask import dataframe as ddf
from dask.diagnostics import ProgressBar
import xesmf as xe
from parallel_pandas import ParallelPandas
#import matplotlib.pyplot as plt
#import cartopy.crs as ccrs
import warnings
warnings.filterwarnings('always')
ParallelPandas.initialize(n_cpu=90, split_factor=45, disable_pr_bar=True)


/home/ec2-user/anaconda3/envs/xesfm/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
    def getFilesForProcessing(path:str) -> pd.DataFrame:
        _listFiles = os.listdir(path)
        _dfFiles = pd.DataFrame(_listFiles,columns=['files'])
        _dfFiles['path']  = path
        _dfFiles['fullNamePath'] = _dfFiles['path']+_dfFiles['files']
        return _dfFiles.drop(columns=['files','path'])

    def interpolateResolution(filePath:str
                               , interpolationMethod:str ='linear'
                               , destinationPath:str=None
                              ):
        global standardizedResolutionParameter
        _fileName = filePath.split('/')[-1]
        _data = xr.open_dataset(filePath)
        _data = _data.interp(lat=np.arange(-90, 90, 0.05)
                                ,lon = np.arange(-180, 180, 0.05)
                             ,method=interpolationMethod)
        _data.to_netcdf(destinationPath+_fileName)


    def regridder(filePath:str
                   , interpolationMethod:str ='linear'
                    , destinationPath:str=None):
        global regridderFile
        global standardizedResolutionParameter
        _fileName = filePath.split('/')[-1]
        _data = xr.open_dataset(filePath)
        regridder = xe.Regridder(_data, standardizedResolutionParameter
                                 , "conservative"
                                 ,weights=regridderFile
                                )
        _data = regridder(_data)
        _data.to_netcdf(destinationPath+_fileName)
        return 0

    standardizedResolutionParameter =   xr.Dataset(
        {
            "lat": (["lat"], np.arange(-90, 90, 0.05)),
            "lon": (["lon"], np.arange(-180, 180, 0.05)),
        }
    )

In [ ]:
sourcePath = 'data/iron/'
destinationPath = 'data/brozen/'
sampleData = 'data/originalnc/Solar-Radiation-Flux_C3S-glob-agric_AgERA5_19860304_final-v1.0.nc'


lat =  np.arange(-90, 90, 0.05)
lon =  np.arange(-180, 180, 0.05)
regridderFile = xr.open_dataset('conservative_1801x3600_3600x7200.nc')
filesForProcess = getFilesForProcessing('data/originalnc/')
filesForProcessDask = ddf.from_pandas(filesForProcess
                                  ,npartitions = 400)

filesForProcess.fullNamePath.apply(lambda f: regridder(filePath=f,destinationPath=destinationPath))

/home/ec2-user/anaconda3/envs/xesfm/lib/python3.12/site-packages/xesmf/backend.py:56: UserWarning: Latitude is outside of [-90, 90]
  warnings.warn('Latitude is outside of [-90, 90]')
/home/ec2-user/anaconda3/envs/xesfm/lib/python3.12/site-packages/xesmf/backend.py:56: UserWarning: Latitude is outside of [-90, 90]
  warnings.warn('Latitude is outside of [-90, 90]')
/home/ec2-user/anaconda3/envs/xesfm/lib/python3.12/site-packages/xesmf/backend.py:56: UserWarning: Latitude is outside of [-90, 90]
  warnings.warn('Latitude is outside of [-90, 90]')
/home/ec2-user/anaconda3/envs/xesfm/lib/python3.12/site-packages/xesmf/backend.py:56: UserWarning: Latitude is outside of [-90, 90]
  warnings.warn('Latitude is outside of [-90, 90]')
/home/ec2-user/anaconda3/envs/xesfm/lib/python3.12/site-packages/xesmf/backend.py:56: UserWarning: Latitude is outside of [-90, 90]
  warnings.warn('Latitude is outside of [-90, 90]')
/home/ec2-user/anaconda3/envs/xesfm/lib/python3.12/site-packages/xesmf/backend.p

In [ ]:
if __name__ == "__main__":
    sourcePath = 'data/iron/'
    destinationPath = 'data/brozen/'
    sampleData = 'data/originalnc/Solar-Radiation-Flux_C3S-glob-agric_AgERA5_19860304_final-v1.0.nc'


    lat =  np.arange(-90, 90, 0.05)
    lon =  np.arange(-180, 180, 0.05)
    regridderFile = xr.open_dataset('conservative_1801x3600_3600x7200.nc')
    filesForProcess = getFilesForProcessing('data/originalnc/')
    filesForProcessDask = ddf.from_pandas(filesForProcess
                                      ,npartitions = 400)
   
    filesForProcessDask.map_partitions(
        lambda df:df.fullNamePath.apply(lambda f: regridder(filePath=f
                                                                 ,destinationPath=destinationPath))
        , meta=('name','i8')
    ).compute(scheduler='processes')

In [6]:
filesForProcessDask

,fullNamePath
npartitions=50,
0,string
315,...
...,...
15392,...
15705,...
